<h1>EXTRACT TRANSFORM LOAD</h1>
EXTRACT - выгружаем данные для обработки<br>
Transform - смесь клининга и ранглинга<br>
LOAD - обработанные данные загружаем в другое место(екстрактим с одного - загркжаем в другое)<br>
в ETL не доходим до анализа. Мы подготавливаем "сырые" данные для анализа<br>
ETL goal is to take data that might be unstructured or difficult to use or access and serve s source of clean, structured data.

<h2 style="color: blue">хорошо работать небольшими кусками данными, лоадидь их, чтобы сильно не нагружало память(дольше будет работатьЬ), также если будет ошибка где-то в середине - придется все делать заново если кусками не лоадидь</h2>

In [2]:
import pandas as pd
import requests

<h1>EXTRACT</h1>

In [3]:
response = requests.get("https://pokeapi.co/api/v2/pokemon/").json()

In [4]:
response

{'count': 1154,
 'next': 'https://pokeapi.co/api/v2/pokemon/?offset=20&limit=20',
 'previous': None,
 'results': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
  {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
  {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
  {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
  {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
  {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
  {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
  {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
  {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
  {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
  {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
  {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
  {'name': 'weedle', '

In [5]:
pokemons = []

for item in response["results"]:
    pokemon_response = requests.get(item["url"]).json()
    pokemons.append(pokemon_response)

In [6]:
df = pd.DataFrame.from_dict(pokemons)
df = df.set_index("id")

In [7]:
df.head(2)

,abilities,base_experience,forms,game_indices,height,held_items,is_default,location_area_encounters,moves,name,order,past_types,species,sprites,stats,types,weight
id,,,,,,,,,,,,,,,,,
1,"[{'ability': {'name': 'overgrow', 'url': 'http...",64,"[{'name': 'bulbasaur', 'url': 'https://pokeapi...","[{'game_index': 153, 'version': {'name': 'red'...",7,[],True,https://pokeapi.co/api/v2/pokemon/1/encounters,"[{'move': {'name': 'razor-wind', 'url': 'https...",bulbasaur,1,[],"{'name': 'bulbasaur', 'url': 'https://pokeapi....",{'back_default': 'https://raw.githubuserconten...,"[{'base_stat': 45, 'effort': 0, 'stat': {'name...","[{'slot': 1, 'type': {'name': 'grass', 'url': ...",69
2,"[{'ability': {'name': 'overgrow', 'url': 'http...",142,"[{'name': 'ivysaur', 'url': 'https://pokeapi.c...","[{'game_index': 9, 'version': {'name': 'red', ...",10,[],True,https://pokeapi.co/api/v2/pokemon/2/encounters,"[{'move': {'name': 'swords-dance', 'url': 'htt...",ivysaur,2,[],"{'name': 'ivysaur', 'url': 'https://pokeapi.co...",{'back_default': 'https://raw.githubuserconten...,"[{'base_stat': 60, 'effort': 0, 'stat': {'name...","[{'slot': 1, 'type': {'name': 'grass', 'url': ...",130


In [8]:
df_columns = ["name", "base_experience", "height", "weight", "types"]

In [9]:
df = df.loc[:, df_columns].copy()
df.head(3)

,name,base_experience,height,weight,types
id,,,,,
1,bulbasaur,64,7,69,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
2,ivysaur,142,10,130,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
3,venusaur,263,20,1000,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."


In [17]:
type_list = [  # убираем и-ю о слотах 
    [slot["type"] for slot in type_]
     for type_ in df["types"]  # испольлзуется фор для облегчения демонстрации примера, но в целом лучше не юзать его
]
type_list[:3]

[[{'name': 'grass', 'url': 'https://pokeapi.co/api/v2/type/12/'},
  {'name': 'poison', 'url': 'https://pokeapi.co/api/v2/type/4/'}],
 [{'name': 'grass', 'url': 'https://pokeapi.co/api/v2/type/12/'},
  {'name': 'poison', 'url': 'https://pokeapi.co/api/v2/type/4/'}],
 [{'name': 'grass', 'url': 'https://pokeapi.co/api/v2/type/12/'},
  {'name': 'poison', 'url': 'https://pokeapi.co/api/v2/type/4/'}]]

In [22]:
flat_types_list = [slot["type"] for type_ in df["types"] for slot in type_] # раскрыл список списков к-й сверху 

In [23]:
flat_types_list[:3]

[{'name': 'grass', 'url': 'https://pokeapi.co/api/v2/type/12/'},
 {'name': 'poison', 'url': 'https://pokeapi.co/api/v2/type/4/'},
 {'name': 'grass', 'url': 'https://pokeapi.co/api/v2/type/12/'}]

In [26]:
df_types = pd.DataFrame.from_records(flat_types_list).drop_duplicates()
df_types

,name,url
0,grass,https://pokeapi.co/api/v2/type/12/
1,poison,https://pokeapi.co/api/v2/type/4/
6,fire,https://pokeapi.co/api/v2/type/10/
9,flying,https://pokeapi.co/api/v2/type/3/
10,water,https://pokeapi.co/api/v2/type/11/
13,bug,https://pokeapi.co/api/v2/type/7/
23,normal,https://pokeapi.co/api/v2/type/1/


In [28]:
all_types = [type_["name"]
    for type_ in pokemon_types
    for pokemon_types in types_list
]

NameError: name 'pokemon_types' is not defined